In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_450x32x32_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_450x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train), to_tf(valid)
#train, valid = to_tf(train).cache(f'{tmp}/ours_32'), to_tf(valid).cache(f'{tmp}/ours_32')

In [3]:
m3 = M_3()
m3.compile(optimizer='Nadam', loss='mae')
m3.build(input_shape=(None, 450, 32, 32, 3))
print(f'Flops per frame:{get_flops(m3, input_sig=[tf.TensorSpec([1, 450, 32, 32, 3])])/450:.0f}')
m3.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:531468
Model: "m_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 450, 64)           196672    
_________________________________________________________________
spectral_transform (Spectral multiple                  82560     
_________________________________________________________________
spectral_transform_1 (Spectr multiple                  49792     
_________________________________________________________________
sequential_3 (Sequential)    (None, 450)               65        
Total params: 329,089
Trainable params: 328,577
Non-trainable params: 512
_________________________________________________________________


In [4]:
m3.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m3_32.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m3.load_weights('../weights/m3_32.h5')

Epoch 1/10
1446/1446 [==============================] - 454s 302ms/step - loss: 0.7886 - val_loss: 0.7311

Epoch 00001: val_loss improved from inf to 0.73110, saving model to ../weights\m3_32.h5
Epoch 2/10
1446/1446 [==============================] - 457s 306ms/step - loss: 0.6653 - val_loss: 0.7249

Epoch 00002: val_loss improved from 0.73110 to 0.72494, saving model to ../weights\m3_32.h5
Epoch 3/10
1446/1446 [==============================] - 451s 302ms/step - loss: 0.6114 - val_loss: 0.6923

Epoch 00003: val_loss improved from 0.72494 to 0.69227, saving model to ../weights\m3_32.h5
Epoch 4/10
1446/1446 [==============================] - 462s 309ms/step - loss: 0.5798 - val_loss: 0.6781

Epoch 00004: val_loss improved from 0.69227 to 0.67811, saving model to ../weights\m3_32.h5
Epoch 5/10
1446/1446 [==============================] - 467s 312ms/step - loss: 0.5696 - val_loss: 0.6712

Epoch 00005: val_loss improved from 0.67811 to 0.67120, saving model to ../weights\m3_32.h5
Epoch 6/1

In [5]:
m3.load_weights('../weights/m3_32.h5')

In [10]:
eval_on_dataset(test_set_CCNU, m3, 450, (32, 32), step=1, batch=32, save='../results/ours_32_st_only_CCNU_CCNU.h5')
get_metrics('../results/ours_32_st_only_CCNU_CCNU.h5', use_filter=False)

100%|██████████| 179/179 [01:46<00:00,  1.68it/s]


{'Sliding window': {'MAE': 4.096, 'RMSE': 13.578, 'R': 0.48057},
 'Whole video': {'MAE': 2.388, 'RMSE': 10.047, 'R': 0.59861}}

In [9]:
eval_on_dataset(test_set_CCNU_rPPG, m3, 450, (32, 32), step=1, batch=32, save='../results/ours_32_st_only_CCNU_CCNU_rPPG.h5')
get_metrics('../results/ours_32_st_only_CCNU_CCNU_rPPG.h5', use_filter=False)

100%|██████████| 54/54 [00:23<00:00,  2.31it/s]


{'Sliding window': {'MAE': 2.813, 'RMSE': 9.59, 'R': 0.63056},
 'Whole video': {'MAE': 1.308, 'RMSE': 4.364, 'R': 0.91283}}

In [8]:
eval_on_dataset(test_set_PURE, m3, 450, (32, 32), step=1, batch=32, save='../results/ours_32_st_only_CCNU_PURE.h5')
get_metrics('../results/ours_32_st_only_CCNU_PURE.h5', use_filter=False)

100%|██████████| 59/59 [00:14<00:00,  4.02it/s]


{'Sliding window': {'MAE': 0.591, 'RMSE': 2.05, 'R': 0.99602},
 'Whole video': {'MAE': 0.529, 'RMSE': 1.037, 'R': 0.99905}}

In [7]:
eval_on_dataset(test_set_UBFC_rPPG2, m3, 450, (32, 32), step=1, batch=32, save='../results/ours_32_st_only_CCNU_UBFC.h5')
get_metrics('../results/ours_32_st_only_CCNU_UBFC.h5', use_filter=False)

100%|██████████| 42/42 [00:09<00:00,  4.37it/s]


{'Sliding window': {'MAE': 1.394, 'RMSE': 3.63, 'R': 0.97793},
 'Whole video': {'MAE': 0.957, 'RMSE': 1.554, 'R': 0.99673}}